# RL Approaches to the Iterated Prisoner's Dilemma

This project demonstrates how simple reinforcement learning algorithms can be used to approach the iterated prisoner's dilemma problem. It is inspired by [this popular Veritasium video](https://www.youtube.com/watch?v=mScpHTIi-kM).

Collaborators:
- Nate Bernich
- Andrew Martin
- Ben Borges
- Jonathan Ranger

*Submitted for CS 751 - Reinforcement Learning at UNH*

## MDP Modeling

The iterated prisoner's dilemma involves two opponents playing the prisoner's dilemma for a fixed number of rounds, where each opponent aims to maximize their own cumulative score.

We will test different modelings of the problem, and use value iteration to compute a policy that will play against known strategies.

Two things will always be the same for our purposes:
- The two actions, to cooperate and defect
- The reward matrix, which is shown below

In [254]:
import statistics
import itertools
import random

actions = ["C", "D"]  # Cooperate, Defect
rewards = {
    "CC": (3, 3), # We cooperate, they cooperate, both earn 3 points
    "CD": (0, 5), # We cooperate, they defect, they earn 5 points, we earn 0
    "DC": (5, 0), # etc...
    "DD": (1, 1),
}

## Previous-Round States

We will start by considering a state space where each state represents the most recent action taken by the player and the opponent strategy. This gives us four states to work with.

The new state in a transition is therefore formed from the chosen actions, and the reward is based on those actions.

In [255]:
states = [a1 + a2 for a1 in actions for a2 in actions]

def transition(state, action, opponent_action):
    return action + opponent_action


def reward(state, action, opponent_action):
    return rewards[action + opponent_action][0]

We will also define some utilities for printing value functions, policies, and the results of a simulated game.

In [256]:
def print_v(v):
  print("Value function:")
  for state in states:
    print(f"State {state} | Value {v[state]:.4f}")

def print_policy(policy):
  print("Policy:")
  for state in states:
    print(f"State {state} | Action {policy[state]}")

def print_game(player_points, opponent_points, player_hist, opponent_hist):
  print("Game results:")
  print(f"Player: {player_points} points")
  print(player_hist)
  print()
  print(f"Opponent: {opponent_points} points")
  print(opponent_hist)

### Tit-for-tat Training

We will now train a policy against an opponent strategy, tit-for-tat.

This strategy has been shown to perform quite well against other strategies. It will retaliate once after every defection by the player (an eye for an eye).

In [257]:
def tit_for_tat(state):
    return state[0] # repeat player's action

We can perform value iteration to determine the values of each state and the optimal action we should take at each one when playing against tit-for-tat.

We see that cooperating is best against this opponent (considering only the previous actions taken at any given time), since this will avoid retaliation.

In [258]:
def compute_policy(opponent_strategy, iterations=1000, discount_factor=0.95):
  v_opt = {state: 0 for state in states}
  policy_opt = {state: None for state in states}

  for _ in range(iterations):
    new_v_opt = v_opt.copy()
    for state in states:

      # Find which action at this state maximizes value
      max_value = float("-inf")
      best_action = None

      for player_action in actions:
        opponent_action = opponent_strategy(state)
        next_state = transition(state, player_action, opponent_action) # next state comes from our action and the opponent's

        immediate_reward = reward(state, player_action, opponent_action) # comes from the reward matrix
        value = immediate_reward + discount_factor * v_opt[next_state]

        if value > max_value:
          max_value = value
          best_action = player_action

      new_v_opt[state] = max_value
      policy_opt[state] = best_action

    v_opt = new_v_opt

  return v_opt, policy_opt

v_opt, policy_opt = compute_policy(opponent_strategy=tit_for_tat)
print_v(v_opt)
print()
print_policy(policy_opt)

Value function:
State CC | Value 60.0000
State CD | Value 60.0000
State DC | Value 57.0000
State DD | Value 57.0000

Policy:
State CC | Action C
State CD | Action C
State DC | Action C
State DD | Action C


We can play a sample game to see how this performs in practice.

We will assume that both our policy and the opponent strategy begin with the assumption that they both would have cooperated before the game. We provide an option to customize this "initial history" which decides the first round of the actual game. As we will show, this will be better than picking arbitrary first actions when we want to test for specific results.

In [259]:
def play_game(policy, opponent_strategy, initial_player_hist="C", initial_opponent_hist="C", rounds=50):
  player_hist = initial_player_hist[-1] # only allow one round in history to be included
  opponent_hist = initial_opponent_hist[-1]
  player_points = 0
  opponent_points = 0

  for _ in range(rounds):
    last_moves = player_hist[-1] + opponent_hist[-1]

    player_action = policy[last_moves]
    opponent_action = opponent_strategy(last_moves)

    reward = rewards[player_action + opponent_action]
    player_points += reward[0]
    opponent_points += reward[1]

    player_hist += player_action
    opponent_hist += opponent_action

  return player_points, opponent_points, player_hist[1:], opponent_hist[1:] # exclude initial history from results

results = play_game(
  policy=policy_opt,
  opponent_strategy=tit_for_tat
)
print_game(*results)

Game results:
Player: 150 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

Opponent: 150 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


(Note: We should really defect in the last round, but since the policy is unaware of when the last round is, this is not expected behavior)

Even if we defect first and trigger one retaliation, our policy rights itself to prevent future retaliation, even though it will lose points in the first actual round. This seems like what we would expect from a sensible solution.

In [260]:
results = play_game(
    policy=policy_opt,
    opponent_strategy=tit_for_tat,
    initial_player_hist="D",
    initial_opponent_hist="C"
)
print_game(*results)

Game results:
Player: 147 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

Opponent: 152 points
DCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


### Discount Factor Adjustment

We observe that with a very low discount factor (such that future rewards are negligible), defecting is advantageous. This makes sense since we don't have to worry about retaliation if the future is not weighed highly.

In [261]:
v_disc_future, policy_disc_future = compute_policy(opponent_strategy=tit_for_tat, discount_factor=0.1)

print_v(v_disc_future)
print()
print_policy(policy_disc_future)

Value function:
State CC | Value 5.1111
State CD | Value 5.1111
State DC | Value 1.1111
State DD | Value 1.1111

Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D


As the discount factor increases from $\gamma = 0.1$ up to $\gamma = 1.0$, we become more cooperative as expected.

In [262]:
for i in range(11):
    gamma = i / 10 # from 1/10 to 10/10
    _, policy_disc_future = compute_policy(discount_factor=gamma, opponent_strategy=tit_for_tat)
    print(f"gamma = {gamma}")
    print_policy(policy_disc_future)
    print()

gamma = 0.0
Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D

gamma = 0.1
Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D

gamma = 0.2
Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D

gamma = 0.3
Policy:
State CC | Action D
State CD | Action D
State DC | Action C
State DD | Action C

gamma = 0.4
Policy:
State CC | Action D
State CD | Action D
State DC | Action C
State DD | Action C

gamma = 0.5
Policy:
State CC | Action D
State CD | Action D
State DC | Action C
State DD | Action C

gamma = 0.6
Policy:
State CC | Action D
State CD | Action D
State DC | Action C
State DD | Action C

gamma = 0.7
Policy:
State CC | Action C
State CD | Action C
State DC | Action C
State DD | Action C

gamma = 0.8
Policy:
State CC | Action C
State CD | Action C
State DC | Action C
State DD | Action C

gamma = 0.9
Policy:
State CC | Action C
State CD | Action C
State DC | Action C
State DD | 

The interesting result is around the middle area, $\gamma = 0.3$ to $\gamma = 0.6$. We defect after our cooperation to gain an immediate win against tit-for-tat in the next round. However, we also sacrifice the next round after our defection by cooperating, in order to right the course and make tit-for-tat cooperate with us in the long term. Thus, the middle-ground discount factors seem to favor contraditory goals that result in a constant back-and-forth with the opponent strategy.

In [263]:
_, policy_mid_discount = compute_policy(discount_factor=0.3, opponent_strategy=tit_for_tat)
print_policy(policy_mid_discount)
print()

results = play_game(policy=policy_mid_discount, opponent_strategy=tit_for_tat)
print_game(*results)

Policy:
State CC | Action D
State CD | Action D
State DC | Action C
State DD | Action C

Game results:
Player: 125 points
DCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDC

Opponent: 125 points
CDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCD


### Arbitrary Action Opponents

Playing against opponent strategies with arbitrary action choices, the value function and policies are varied. However, our player is significantly better on average and is mostly able to learn the strategy well enough to earn singificant point totals. This indicates some basic success of our RL approach in its current form.

Note that the goal is not to beat the opponent's score, but to maximize points in general. It just happens that winning more points often involves taking advantage of the opponent. So, we argue that the high difference between our policy's points and the opponent strategies' points can be considered a good indicator for success.

In [264]:
def score_random_games(game_count=500, discount_factor=0.95, training_iterations=500):
  player_results = []
  opponent_results = []
  result_diffs = []

  for i in range(game_count):

    arbitrary_policy = {state: random.choice(actions) for state in states}
    
    def arbitrary_strategy(state):
      return arbitrary_policy[state]

    _, policy = compute_policy(
        iterations=training_iterations,
        discount_factor=discount_factor,
        opponent_strategy=arbitrary_strategy
      )

    player_points, opponent_points, _, _ = play_game(policy, arbitrary_strategy)

    if i < 10:
      print(f"Game {i}: {player_points} - {opponent_points}")

    player_results.append(player_points)
    opponent_results.append(opponent_points)
    result_diffs.append(player_points - opponent_points)


  print("...")
  print(f"Overall statistics for {game_count} games played:")
  print(f"Average player points: {statistics.mean(player_results)}")
  print(f"Average opponent points: {statistics.mean(opponent_results)}")
  print(f"Average difference (player - opponent): {statistics.mean(result_diffs)}")

score_random_games()

Game 0: 150 - 25
Game 1: 250 - 0
Game 2: 150 - 25
Game 3: 50 - 50
Game 4: 246 - 1
Game 5: 50 - 50
Game 6: 250 - 0
Game 7: 50 - 50
Game 8: 125 - 125
Game 9: 50 - 50
...
Overall statistics for 500 games played:
Average player points: 172.874
Average opponent points: 43.494
Average difference (player - opponent): 129.38


### Pacifist Opponent

When playing against a pacifist opponent, we will always defect. This gives us the maximum number of points possible.

In [265]:
def pacifist(state):
    return "C"

v_pacifist, policy_pacifist = compute_policy(opponent_strategy=pacifist)

print_v(v_pacifist)
print()
print_policy(policy_pacifist)

print()
pacifist_results = play_game(policy_pacifist, pacifist)
print_game(*pacifist_results)

Value function:
State CC | Value 100.0000
State CD | Value 100.0000
State DC | Value 100.0000
State DD | Value 100.0000

Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D

Game results:
Player: 250 points
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD

Opponent: 0 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


### Aggressor Opponent

When playing against an opponent who always defects, we will always defect since we have no better option. Cooperating will earn the player far fewer points since the opponent would take advantage of them every time.

In [266]:
def agressor(state):
    return "D"

v_aggressor, policy_aggressor = compute_policy(opponent_strategy=agressor)

print_v(v_aggressor)
print()
print_policy(policy_aggressor)

print()
results_aggressor = play_game(policy_aggressor, agressor)
print_game(*results_aggressor)

Value function:
State CC | Value 20.0000
State CD | Value 20.0000
State DC | Value 20.0000
State DD | Value 20.0000

Policy:
State CC | Action D
State CD | Action D
State DC | Action D
State DD | Action D

Game results:
Player: 50 points
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD

Opponent: 50 points
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD


### Friedman

Another common and interesting opponent strategy is called Friedman. This strategy will cooperate until the player defects once. Then, it will defect the rest of the game.

We have devised a way to implement this using states that contain only the previous round:

1. If the player defected last, we defect.

2. If we defected before, defect again. At some point, our repeat defections must have started and we need to continue them.

3. Otherwise, cooperate.

In [267]:
def friedman(state):
    if state[0] == "D":
        return "D"

    if state[1] == "D":
        return "D"

    return "C"

v_friedman, policy_friedman = compute_policy(opponent_strategy=friedman)

print_v(v_friedman)
print()
print_policy(policy_friedman)

Value function:
State CC | Value 60.0000
State CD | Value 20.0000
State DC | Value 20.0000
State DD | Value 20.0000

Policy:
State CC | Action C
State CD | Action D
State DC | Action D
State DD | Action D


If we assume cooperation initially, Friedman will cooperate with us all the way through.

In [268]:
friedman_results = play_game(
    policy_friedman,
    friedman,
    initial_player_hist="C",
    initial_opponent_hist="C"
)
print_game(*friedman_results)

Game results:
Player: 150 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC

Opponent: 150 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


However, if we defect, Friedman will defect the rest of the way too. We respond by defecting for the same reason considered when playing against the aggressor strategy. So, it seems our policy is making the right decisions.

In [269]:
friedman_results_2 = play_game(
    policy_friedman,
    friedman,
    initial_player_hist="D",
    initial_opponent_hist="C"
)
print_game(*friedman_results_2)

Game results:
Player: 50 points
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD

Opponent: 50 points
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD


### Joss

The algorithm for Joss is the following:

1. Defect `10%` of the time.

2. Otherwise, copy the player's last move.

In [270]:
def joss(state):
    if random.random() < 0.1:
        return "D" # sneaky move
    
    return state[-1][0] # copies the player

It seems we usually beat or tie Joss. However, subsequent runs prove that the results vary drastically.

This exposes a potential weakness when faced against strategies that involve random actions which may be different from the random actions that we trained on.

In [271]:
v_joss, policy_joss = compute_policy(opponent_strategy=joss)

print_v(v_joss)
print()
print_policy(policy_joss)
print()

joss_results = play_game(
    policy_joss,
    joss,
    initial_player_hist="C",
    initial_opponent_hist="C"
)
print_game(*joss_results)

Value function:
State CC | Value 95.3457
State CD | Value 20.0000
State DC | Value 97.3457
State DD | Value 20.0000

Policy:
State CC | Action C
State CD | Action D
State DC | Action D
State DD | Action D

Game results:
Player: 105 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCDDDDDDDDDDDDDDDDDDDDD

Opponent: 110 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCDDDDDDDDDDDDDDDDDDDDDD


## $k$-History

We next consider what would happen when we expand the state space by representing the most recent $k$ rounds (actions from both the player and opponent) in the state instead of the single most recent.

This enables us to experiment with different strategies that look back further than one round. We call this "$k$-history" and our examples will use $k = 3$.

In [272]:
k = 3
possible_rounds = ["".join(perm) for perm in itertools.product(actions, repeat=2)]
k_hist_states = list(itertools.product(possible_rounds, repeat=k))

print(f"Number of states for k = {k}: {len(k_hist_states)}")
for k_hist_state in k_hist_states:
    print(k_hist_state)

Number of states for k = 3: 64
('CC', 'CC', 'CC')
('CC', 'CC', 'CD')
('CC', 'CC', 'DC')
('CC', 'CC', 'DD')
('CC', 'CD', 'CC')
('CC', 'CD', 'CD')
('CC', 'CD', 'DC')
('CC', 'CD', 'DD')
('CC', 'DC', 'CC')
('CC', 'DC', 'CD')
('CC', 'DC', 'DC')
('CC', 'DC', 'DD')
('CC', 'DD', 'CC')
('CC', 'DD', 'CD')
('CC', 'DD', 'DC')
('CC', 'DD', 'DD')
('CD', 'CC', 'CC')
('CD', 'CC', 'CD')
('CD', 'CC', 'DC')
('CD', 'CC', 'DD')
('CD', 'CD', 'CC')
('CD', 'CD', 'CD')
('CD', 'CD', 'DC')
('CD', 'CD', 'DD')
('CD', 'DC', 'CC')
('CD', 'DC', 'CD')
('CD', 'DC', 'DC')
('CD', 'DC', 'DD')
('CD', 'DD', 'CC')
('CD', 'DD', 'CD')
('CD', 'DD', 'DC')
('CD', 'DD', 'DD')
('DC', 'CC', 'CC')
('DC', 'CC', 'CD')
('DC', 'CC', 'DC')
('DC', 'CC', 'DD')
('DC', 'CD', 'CC')
('DC', 'CD', 'CD')
('DC', 'CD', 'DC')
('DC', 'CD', 'DD')
('DC', 'DC', 'CC')
('DC', 'DC', 'CD')
('DC', 'DC', 'DC')
('DC', 'DC', 'DD')
('DC', 'DD', 'CC')
('DC', 'DD', 'CD')
('DC', 'DD', 'DC')
('DC', 'DD', 'DD')
('DD', 'CC', 'CC')
('DD', 'CC', 'CD')
('DD', 'CC', 'DC')


We redefine the value function and policy display utilities to work with this modeling. While the reward remains the same, the transition is a bit different.

In [273]:
def k_hist_print_v(v):
  print("Value function:")
  for k_hist_state in k_hist_states:
    print(f"State {k_hist_state} | Value {v[k_hist_state]:.4f}")

def k_hist_print_policy(policy):
  print("Policy:")
  for k_hist_state in k_hist_states:
    print(f"State {k_hist_state} | Action {policy[k_hist_state]}")

def k_hist_transition(k_hist_state, player_action, opponent_action):
    new_round = player_action + opponent_action
    return k_hist_state[1:] + (new_round,) # most recent two rounds + the new round

def k_hist_reward(player_action, opponent_action):
    return rewards[player_action + opponent_action][0]

### Tit-for-two-tats

We consider a new strategy, tit-for-two-tats. This requires $k \ge 2$ and will defect only if the player defects twice in a row.

In [274]:
def tit_for_two_tats(k_hist_state):
    if k_hist_state[-1][0] == "D" and k_hist_state[-2][0] == "D":
        return "D"
    return "C"

We use a modified value iteration approach to work with $k$-history. The resulting policy is evidently much more complex than the one trained to oppose tit-for-tat.

In [275]:
def k_hist_compute_policy(opponent_strategy, iterations=1000, discount_factor=0.95):
    v_opt = {k_hist_state: 0 for k_hist_state in k_hist_states}
    policy_opt = {k_hist_state: None for k_hist_state in k_hist_states}

    for _ in range(iterations):
        new_v_opt = v_opt.copy()
        for k_hist_state in k_hist_states:

            max_value = float("-inf")
            best_action = None

            for player_action in actions:

                opponent_action = opponent_strategy(k_hist_state)
                next_state = k_hist_transition(k_hist_state, player_action, opponent_action)

                immediate_reward = k_hist_reward(player_action, opponent_action)
                value = immediate_reward + discount_factor * v_opt[next_state]

                if value > max_value:
                    max_value = value
                    best_action = player_action

            new_v_opt[k_hist_state] = max_value
            policy_opt[k_hist_state] = best_action

        v_opt = new_v_opt

    return v_opt, policy_opt

two_tats_v, two_tats_policy = k_hist_compute_policy(
    opponent_strategy=tit_for_two_tats
)

k_hist_print_v(two_tats_v)
print()
k_hist_print_policy(two_tats_policy)

Value function:
State ('CC', 'CC', 'CC') | Value 80.5128
State ('CC', 'CC', 'CD') | Value 80.5128
State ('CC', 'CC', 'DC') | Value 79.4872
State ('CC', 'CC', 'DD') | Value 79.4872
State ('CC', 'CD', 'CC') | Value 80.5128
State ('CC', 'CD', 'CD') | Value 80.5128
State ('CC', 'CD', 'DC') | Value 79.4872
State ('CC', 'CD', 'DD') | Value 79.4872
State ('CC', 'DC', 'CC') | Value 80.5128
State ('CC', 'DC', 'CD') | Value 80.5128
State ('CC', 'DC', 'DC') | Value 76.4872
State ('CC', 'DC', 'DD') | Value 76.4872
State ('CC', 'DD', 'CC') | Value 80.5128
State ('CC', 'DD', 'CD') | Value 80.5128
State ('CC', 'DD', 'DC') | Value 76.4872
State ('CC', 'DD', 'DD') | Value 76.4872
State ('CD', 'CC', 'CC') | Value 80.5128
State ('CD', 'CC', 'CD') | Value 80.5128
State ('CD', 'CC', 'DC') | Value 79.4872
State ('CD', 'CC', 'DD') | Value 79.4872
State ('CD', 'CD', 'CC') | Value 80.5128
State ('CD', 'CD', 'CD') | Value 80.5128
State ('CD', 'CD', 'DC') | Value 79.4872
State ('CD', 'CD', 'DD') | Value 79.4872


We gain a better understanding of the policy by simulating a game. It seems that the policy will defect every other round to gain the maximum number of points in those rounds. However, it never defects twice in a row since this would trigger the revenge mechanism.

In [276]:
def k_hist_play_game(policy, opponent_strategy, initial_player_hist="CCC", initial_opponent_hist="CCC", rounds=50):
    player_hist = initial_player_hist
    opponent_hist = initial_opponent_hist

    player_points = 0
    opponent_points = 0

    for _ in range(rounds):

        k_hist_state = tuple("".join(pair) for pair in zip(player_hist[-k:], opponent_hist[-k:])) # collect the most recent k rounds from game history

        # Player selects action based on policy
        player_action = policy[k_hist_state]

        # Opponent selects action based on their strategy
        opponent_action = opponent_strategy(k_hist_state)

        # Calculate rewards
        reward_pair = rewards[player_action + opponent_action]
        player_points += reward_pair[0]
        opponent_points += reward_pair[1]

        # Update histories
        player_hist += player_action
        opponent_hist += opponent_action

    return player_points, opponent_points, player_hist[3:], opponent_hist[3:] # truncate initial history just as before

# Play the game using the optimal policy against tit-for-tat
two_tats_results = k_hist_play_game(
    two_tats_policy,
    tit_for_two_tats
)
print_game(*two_tats_results)

Game results:
Player: 200 points
DCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDC

Opponent: 75 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


Even when we start with two defections to force the revenge mechanism, our policy will quickly right the course, only losing minimal points.

In [277]:
two_tats_results_2 = k_hist_play_game(
    two_tats_policy,
    tit_for_two_tats,
    initial_player_hist="CDD",
    initial_opponent_hist="CCC"
)
print_game(*two_tats_results_2)

Game results:
Player: 197 points
CDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCDCD

Opponent: 77 points
DCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


### Tit-for-$k$-tats

We can extend the opponent strategy to one we will call "tit-for-$k$-tats". It will only defect when the player defects $k$ times in a row (every round observable in the state).

In [278]:
def tit_for_k_tats(k_hist_state):
    if all(round[0] == "D" for round in k_hist_state):
        return "D"
    return "C"

The resulting policy is a bit different, defecting much more often.

In [279]:
k_tats_v, k_tats_policy = k_hist_compute_policy(
    opponent_strategy=tit_for_k_tats
)

k_hist_print_v(k_tats_v)
print()
k_hist_print_policy(k_tats_policy)

Value function:
State ('CC', 'CC', 'CC') | Value 87.3444
State ('CC', 'CC', 'CD') | Value 87.3444
State ('CC', 'CC', 'DC') | Value 86.6784
State ('CC', 'CC', 'DD') | Value 86.6784
State ('CC', 'CD', 'CC') | Value 87.3444
State ('CC', 'CD', 'CD') | Value 87.3444
State ('CC', 'CD', 'DC') | Value 86.6784
State ('CC', 'CD', 'DD') | Value 86.6784
State ('CC', 'DC', 'CC') | Value 87.3444
State ('CC', 'DC', 'CD') | Value 87.3444
State ('CC', 'DC', 'DC') | Value 85.9772
State ('CC', 'DC', 'DD') | Value 85.9772
State ('CC', 'DD', 'CC') | Value 87.3444
State ('CC', 'DD', 'CD') | Value 87.3444
State ('CC', 'DD', 'DC') | Value 85.9772
State ('CC', 'DD', 'DD') | Value 85.9772
State ('CD', 'CC', 'CC') | Value 87.3444
State ('CD', 'CC', 'CD') | Value 87.3444
State ('CD', 'CC', 'DC') | Value 86.6784
State ('CD', 'CC', 'DD') | Value 86.6784
State ('CD', 'CD', 'CC') | Value 87.3444
State ('CD', 'CD', 'CD') | Value 87.3444
State ('CD', 'CD', 'DC') | Value 86.6784
State ('CD', 'CD', 'DD') | Value 86.6784


As a simulation of the game shows, the player under this policy will again defect as much as they can get away with, without triggering the revenge mechanism. This ultimately shows that $k$-history modeling will allow us to compute policies that effectively counter more complicated opponent strategies.

In [280]:
k_tats_results = k_hist_play_game(
    k_tats_policy,
    tit_for_k_tats,
    initial_player_hist="CCC",
    initial_opponent_hist="CCC"
)
print_game(*k_tats_results)

Game results:
Player: 218 points
DDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDDCDD

Opponent: 48 points
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC


## Conclusion

Our testing has shown that simple RL algorithms produce desirable results when trained against deterministic opponent strategies. Extensions such as $k$-history also allow us to simulate more complex deterministic opponents, which our policies are still able to adequately play against. We only found that opponents with randomness may not allow us to compute an optimal policy. The weakness arises from a disconnect between the random actions chosen in training, and those chosen in a simulation.

We conclude that our approach is an interesting case study for how RL can be utilized effectively in relatively simple problems, and how certain parameters and modeling can be tuned to explore different results.